In [1]:
## CSE 519 - Workload Prediction - Memory

In [2]:
import numpy as np
import pandas as pd
import scipy as sc
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split #training and testing data split
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import VarianceThreshold
from sklearn import preprocessing
from sklearn import utils
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.decomposition import PCA

In [3]:
#1. Load data separated by spaces

In [4]:
#cols=[(1,5),(5,17),(17,25),(25,33),(33,34),(34,37),(37,45),(45,53),(53,57),(453,454),(1367,1371),(566,567)]
headerNames = ["JobID", "SubmitTime", "WaitTime", "RunTime","NProcs", "AverageCPUTimeUsed",
               "UsedMemory","ReqNProcs","ReqTime","ReqMemory","Status","UserID","GroupID","ExecutableID",
               "QueueID","PartitionID","OrigSiteID","LastRunSiteID","JobStructure","JobStructureParams","UsedNetwork",
               "UsedLocalDiskSpace","UsedResources","OrigSiteID","ReqPlatform","ReqNetwork","ReqLocalDiskSpace","ReqResources"
               "VOID", "ProjectID"]
prefix = "/users/tanle/data/"
#prefix = "/data/"
df =pd.read_csv(prefix + "anon_jobs.gwf", header=None, delim_whitespace=True,  names=headerNames, index_col=0,)
df2 =pd.read_csv(prefix + "anon_jobs3.gwf", header=None, delim_whitespace=True,  names=headerNames, index_col=0,)
df = df.append(df2)
#df = df.sample(frac=0.25, replace=True)
df['JobStructure'].unique()

/users/tanle/anaconda3/lib/python3.5/site-packages/pandas/io/parsers.py:705: UserWarning: Duplicate names specified. This will raise an error in the future.
  return _read(filepath_or_buffer, kwds)
/users/tanle/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/users/tanle/anaconda3/lib/python3.5/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


array(['UNITARY', 'BoT', -1], dtype=object)

In [5]:
df.shape

(1906142, 28)

In [6]:
df.columns

Index(['SubmitTime', 'WaitTime', 'RunTime', 'NProcs', 'AverageCPUTimeUsed',
       'UsedMemory', 'ReqNProcs', 'ReqTime', 'ReqMemory', 'Status', 'UserID',
       'GroupID', 'ExecutableID', 'QueueID', 'PartitionID', 'OrigSiteID',
       'LastRunSiteID', 'JobStructure', 'JobStructureParams', 'UsedNetwork',
       'UsedLocalDiskSpace', 'UsedResources', 'OrigSiteID.1', 'ReqPlatform',
       'ReqNetwork', 'ReqLocalDiskSpace', 'ReqResourcesVOID', 'ProjectID'],
      dtype='object')

In [7]:
df.describe()

,SubmitTime,WaitTime,RunTime,NProcs,AverageCPUTimeUsed,UsedMemory,ReqNProcs,ReqTime,ReqMemory,Status,PartitionID,UsedNetwork,UsedLocalDiskSpace,OrigSiteID.1,ReqPlatform,ReqNetwork,ReqLocalDiskSpace,ReqResourcesVOID,ProjectID
count,1.906142e+06,1.906142e+06,1.906142e+06,1.906142e+06,1.906142e+06,1.906142e+06,1.906142e+06,1.906142e+06,1906142.0,1.906142e+06,1906142.0,1906142.0,1906142.0,1906142.0,1906142.0,1906142.0,1906142.0,1906142.0,1906142.0
mean,1.131852e+09,2.547754e+02,3.681352e+04,2.980975e+00,1.968374e+01,1.090208e+05,2.980975e+00,2.746489e+03,-1.0,1.795459e-01,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
std,1.927062e+07,2.769963e+03,1.872258e+05,5.203352e+00,2.374573e+02,3.392265e+05,5.203352e+00,2.180845e+05,0.0,9.834400e-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,1.046680e+09,0.000000e+00,-6.471000e+03,1.000000e+00,-1.000000e+00,-1.000000e+00,1.000000e+00,-1.000000e+00,-1.0,-1.000000e+00,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
25%,1.119819e+09,0.000000e+00,2.800000e+01,1.000000e+00,-1.000000e+00,-1.000000e+00,1.000000e+00,-1.000000e+00,-1.0,-1.000000e+00,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
50%,1.134031e+09,3.000000e+00,1.610000e+02,1.000000e+00,6.000000e-03,4.790272e+03,1.000000e+00,6.600000e+02,-1.0,1.000000e+00,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
75%,1.144612e+09,8.000000e+00,4.376000e+03,2.000000e+00,4.000000e-02,1.304986e+04,2.000000e+00,9.000000e+02,-1.0,1.000000e+00,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
max,1.166002e+09,1.302269e+06,1.807190e+07,1.280000e+02,5.107015e+04,4.294951e+06,1.280000e+02,6.000000e+07,-1.0,1.000000e+00,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [8]:
df['JobStructureNum']  = -1 # default value
df.loc[df['JobStructure'] == 'UNITARY', 'JobStructureNum'] = 1
df.loc[df['JobStructure'] == 'BoT', 'JobStructureNum'] = 2
df['JobStructureNum'].unique()

array([ 1,  2, -1])

In [9]:
df['MemoryGB']  = -1 # default v+alue
df['MemoryGB'] = np.ceil(df['UsedMemory']/(1024*1024))
df['MemoryGB'].unique()
#np.std(df['MemoryGB'])

array([ 1.,  0.,  4.,  3.,  5.,  2.])

In [10]:
# drop some columns
df = df.drop('UsedResources', axis=1)
df = df.drop('JobStructure', axis=1) 

# TODO - refactor the string values to numeric

df = df.drop('LastRunSiteID', axis=1)
df = df.drop('OrigSiteID', axis=1)
df = df.drop('OrigSiteID.1', axis=1)
df = df.drop('QueueID', axis=1)
df = df.drop('ExecutableID', axis=1)
df = df.drop('UserID', axis=1)
df = df.drop('GroupID', axis=1)
df = df.drop('JobStructureParams', axis=1)
df = df.drop('UsedMemory', axis=1)
df = df.drop('ReqMemory', axis=1)


# features that can be dropped because they are either related to other outputs
df = df.drop('WaitTime', axis=1)
df = df.drop('RunTime', axis=1)

df.columns

Index(['SubmitTime', 'NProcs', 'AverageCPUTimeUsed', 'ReqNProcs', 'ReqTime',
       'Status', 'PartitionID', 'UsedNetwork', 'UsedLocalDiskSpace',
       'ReqPlatform', 'ReqNetwork', 'ReqLocalDiskSpace', 'ReqResourcesVOID',
       'ProjectID', 'JobStructureNum', 'MemoryGB'],
      dtype='object')

In [11]:
# check correlation against one or more input variables(features) against the output label(UsedMemory)
#%matplotlib inline
#sns.pairplot(data=df, x_vars=['SubmitTime'], y_vars=['MemoryGB'], size=17, aspect=0.7,kind='reg')

In [12]:
# split the dataset into training and test data
# random_state ensures that same train test split happens everytime if done repeatedly
train, test = train_test_split(df, random_state=42, test_size=0.30)
#type(train)

In [13]:
# extract label from training set - used memory
train_label = train.loc[:, train.columns == 'MemoryGB']

In [14]:
# extract features from training set - all columns except used memory
train_features = train.loc[:, train.columns != 'MemoryGB']
train_features.columns

Index(['SubmitTime', 'NProcs', 'AverageCPUTimeUsed', 'ReqNProcs', 'ReqTime',
       'Status', 'PartitionID', 'UsedNetwork', 'UsedLocalDiskSpace',
       'ReqPlatform', 'ReqNetwork', 'ReqLocalDiskSpace', 'ReqResourcesVOID',
       'ProjectID', 'JobStructureNum'],
      dtype='object')

In [15]:
#extract label from test set
test_label = test.loc[:, test.columns == 'MemoryGB']
test_label.columns

Index(['MemoryGB'], dtype='object')

In [16]:
#extract features from test set
test_features = test.loc[: , test.columns != 'MemoryGB']
test_features.columns

Index(['SubmitTime', 'NProcs', 'AverageCPUTimeUsed', 'ReqNProcs', 'ReqTime',
       'Status', 'PartitionID', 'UsedNetwork', 'UsedLocalDiskSpace',
       'ReqPlatform', 'ReqNetwork', 'ReqLocalDiskSpace', 'ReqResourcesVOID',
       'ProjectID', 'JobStructureNum'],
      dtype='object')

In [17]:
test_features.head(5)

,SubmitTime,NProcs,AverageCPUTimeUsed,ReqNProcs,ReqTime,Status,PartitionID,UsedNetwork,UsedLocalDiskSpace,ReqPlatform,ReqNetwork,ReqLocalDiskSpace,ReqResourcesVOID,ProjectID,JobStructureNum
JobID,,,,,,,,,,,,,,,
516480,1129893178,1,-1.0000,1,-1.0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
272273,1139451924,2,0.0150,2,960.0,1,-1,-1,-1,-1,-1,-1,-1,-1,1
848406,1132654620,2,0.1000,2,900.0,1,-1,-1,-1,-1,-1,-1,-1,-1,1
504477,1163759119,8,0.0025,8,960.0,1,-1,-1,-1,-1,-1,-1,-1,-1,1
476723,1161766243,1,0.0400,1,60000.0,1,-1,-1,-1,-1,-1,-1,-1,-1,1


In [18]:
sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
train_new_features=sel.fit_transform(train_features)
train_new_features.shape


(1334299, 7)

In [19]:
train_new_features[0]

array([  1.13893353e+09,   2.00000000e+00,   2.00000000e-02,
         2.00000000e+00,   9.60000000e+02,   1.00000000e+00,
         1.00000000e+00])

In [20]:
train_features.head(5)

,SubmitTime,NProcs,AverageCPUTimeUsed,ReqNProcs,ReqTime,Status,PartitionID,UsedNetwork,UsedLocalDiskSpace,ReqPlatform,ReqNetwork,ReqLocalDiskSpace,ReqResourcesVOID,ProjectID,JobStructureNum
JobID,,,,,,,,,,,,,,,
262295,1138933534,2,0.020000,2,960.0,1,-1,-1,-1,-1,-1,-1,-1,-1,1
689462,1137123016,7,0.001429,7,960.0,1,-1,-1,-1,-1,-1,-1,-1,-1,1
463105,1124432912,1,-1.000000,1,-1.0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
689375,1141614791,1,-1.000000,1,-1.0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
42505,1093506531,1,-1.000000,1,-1.0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [21]:
# variance threshold applied to test features too
test_new_features = sel.fit_transform(test_features)
test_new_features.shape

(571843, 7)

In [22]:
from sklearn.model_selection import KFold # import KFold
from sklearn.cross_validation import StratifiedKFold

conv_X = pd.get_dummies(train_features.ix[:, :])
conv_Y = pd.get_dummies(train_label.ix[:, :])

#print('numpy.isnan(y)', np.isnan(conv_Y).any())
#print('numpy.isnan(x)', np.isnan(conv_X).any())
#print(conv_X[conv_X['NProcs'] == 78734,:])
#print(type(conv_X))

linreg = LinearRegression()
n_folds = 10

#cv = KFold(len(train), n_splits=10, indices=False)
kf = KFold(n_splits=10)
#skf = StratifiedKFold(train_label, n_folds=n_folds, shuffle=True)

#print("data to split-type", type(train))
#print("data to split", train)
#print("data to split - shape", train.shape)

i = 0 
for train_idx, test_idx in kf.split(train):
    print("Running Fold", i+1, "/", n_folds)
    
    #print("\n after fold", train1,test1)
    #print("\n after fold - shape", train1.shape,test1.shape)
    linreg = LinearRegression()
    
    train_X = conv_X.iloc[train_idx, :]
    #print("\n train_X ", train_X)
    train_Y = conv_Y.iloc[train_idx]
    #print("\n train_Y ", train_X)
    test_X = conv_X.iloc[test_idx, :]
    #print("\n test_X ", train_X)
    test_Y = conv_Y.iloc[test_idx]
    #print("\n test_Y ", train_X)
    
    
    linreg.fit(train_X, train_Y)
    test_pred = linreg.predict(test_X)
    print (np.sqrt(mean_squared_error(test_Y, test_pred))) #Test RMSE
    i+=1
    #linreg.fit(train_features1, train_label1)
    #test_pred1 = linreg.predict(testk_features1)
    #print (np.sqrt(mean_squared_error(test_label, test_pred1))) #Test RMSE

/users/tanle/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/users/tanle/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:4: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
/users/tanle/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/st

Running Fold 1 / 10
0.403064093243
Running Fold 2 / 10
0.400976763079
Running Fold 3 / 10
0.404095444849
Running Fold 4 / 10
0.403167745878
Running Fold 5 / 10
0.403796171266
Running Fold 6 / 10
0.404895415608
Running Fold 7 / 10
0.404392372996
Running Fold 8 / 10
0.403684460253
Running Fold 9 / 10
0.406876908802
Running Fold 10 / 10
0.403637800166


In [23]:

linreg = LinearRegression()
linreg.fit(train_features, train_label)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [24]:
train_pred = linreg.predict(train_features)
print (np.sqrt(mean_squared_error(train_label, train_pred))) # Training RMSE


0.403847440307


In [25]:
test_pred = linreg.predict(test_features)
#test_pred
#test_label

#print (mean_absolute_error(test_label, test_pred)) #MAE
#print(mean_squared_error(test_label, test_pred)) #MSE
print (np.sqrt(mean_squared_error(test_label, test_pred))) #Test RMSE

0.402675052327
